In [5]:
import os
import glob

In [11]:
import pandas as pd

In [6]:
data_dir = '/data/hts/2018/foot'

In [108]:
fs = glob.glob(os.path.join(data_dir, '*.tab'))

In [80]:
from functools import reduce

In [111]:
d = {}

for f in fs[0:]:
    path, filename = os.path.split(f)
    name, ext = os.path.splitext(filename)
    sid, method, person, _, lane, _, _ = name.split('_')
    d.setdefault((sid, method, person), []).append(filename)

In [207]:
dfs = []
for key in d:
    dfs_ = []
    for filename in sorted(d[key]):
        df_ = pd.read_table(os.path.join(data_dir, filename), header=None, skiprows=4)
        df_.columns = ['gene', 'a', 'b', 'c']
        df_ = df_.iloc[:, [0,3]]
        dfs_.append(df_)
    sid, method, person = key
    df = reduce(lambda left,right: pd.merge(left, right, on='gene'), dfs_)
    df['count'] = df.sum(axis=1)
    df['sid'] = sid
    df = df[['sid', 'gene', 'count']]
    df = df.pivot('sid', 'gene', 'count')
    df['person'] = person
    df['method'] = method
    df.reset_index(level=0, inplace=True)
    dfs.append(df)
df_all = pd.concat(dfs)

In [208]:
df_all.columns

Index(['sid', 'gene0', 'gene1', 'gene10', 'gene100', 'gene1000', 'gene1001',
       'gene1002', 'gene1003', 'gene1004',
       ...
       'gene992', 'gene993', 'gene994', 'gene995', 'gene996', 'gene997',
       'gene998', 'gene999', 'person', 'method'],
      dtype='object', name='gene', length=8328)

In [209]:
df_all = df_all[df_all.columns[0:1].tolist() + df_all.columns[-2:] .tolist() + df_all.columns[1:-2].tolist()]

In [216]:
df_all['sid'] = df_all['sid'].astype('int')

In [217]:
df_all = df_all.sort_values(['sid', 'person', 'method'])

In [218]:
df_all.to_csv('gene_counts.txt', sep='\t', index=False)